1. Colab 런타임 준비
GPU 활성화

메뉴 > 런타임 > 런타임 유형 변경에서 하드웨어 가속기: GPU 선택

구글 드라이브 마운트 (선택 사항)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. HRNet-Image-Classification 레포지토리 클론 및 의존성 설치

In [2]:
# 레포지토리 클론
!git clone https://github.com/HRNet/HRNet-Image-Classification.git
%cd HRNet-Image-Classification

# 필요 패키지 설치 (torch, torchvision 등)
!pip install -r requirements.txt

# Hugging Face Datasets 설치
!pip install datasets

Cloning into 'HRNet-Image-Classification'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 169 (delta 52), reused 41 (delta 41), pack-reused 104 (from 1)
Receiving objects: 100% (169/169), 186.75 KiB | 1017.00 KiB/s, done.
Resolving deltas: 100% (94/94), done.
/content/HRNet-Image-Classification
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Could not find a version that satisfies the requirement opencv-python==3.4.1.15 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86)
ERROR: No matching distribution found for opencv-python=

In [5]:
!pip install tensorboardX
!pip install yacs
!pip install _init_paths



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.4 MB/s eta 0:00:00
ERROR: Invalid requirement: '_init_paths': Expected package name at the start of dependency specifier
    _init_paths
    ^


3. 데이터셋 로딩 및 분류용 전처리
Hugging Face segmentation 데이터셋을 로드한 뒤, 마스크 유무로 0/1 레이블을 만든 뒤 로컬에 ImageFolder 구조로 저장합니다.

In [4]:
#데이터셋 증강 적용
from datasets import load_dataset
from PIL import Image
import os

# 1) 원본 HF 데이터셋 로드
#    train · 7.51k, test · 1.08k, valid · 2.15k rows
ds = load_dataset("espada105/augmented-brain-tumor-segmentation-v2")

# 2) 저장할 폴더 구조 생성 (이미 train/val 폴더가 만들어져 있다면 test만 추가)
base = "/content/HRNet-Image-Classification/brain_tumor_classification/imagenet/images"
for phase in ("train","val","test"):
    for cls in ("no_tumor","tumor"):
        os.makedirs(f"{base}/{phase}/{cls}", exist_ok=True)

# 3) split별로 이미지 저장 함수
def save_split(split_ds, phase):
    for i, item in enumerate(split_ds):
        img: Image.Image = item["image"]
        cid = int(item["category_id"])
        # (1=no_tumor, 2=tumor)
        label = "tumor"    if cid == 2 else \
                "no_tumor" if cid == 1 else None
        if label is None:
            continue
        img.save(f"{base}/{phase}/{label}/{i}.png")

# 4) train/val/test 각각 뽑아서 저장
save_split(ds["train"], "train")
save_split(ds["valid"], "val")
save_split(ds["test"],  "test")

# 5) 결과 확인
print("Train/Tumor   :", len(os.listdir(f"{base}/train/tumor")))
print("Train/NoTumor :", len(os.listdir(f"{base}/train/no_tumor")))
print("Val/Tumor     :", len(os.listdir(f"{base}/val/tumor")))
print("Val/NoTumor   :", len(os.listdir(f"{base}/val/no_tumor")))
print("Test/Tumor    :", len(os.listdir(f"{base}/test/tumor")))
print("Test/NoTumor  :", len(os.listdir(f"{base}/test/no_tumor")))


README.md:   0%|          | 0.00/814 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/408M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/407M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/404M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/407M [00:00<?, ?B/s]

...

valid-00000-of-00001.parquet:   0%|          | 0.00/462M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7510 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1075 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/2145 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [5]:
#원본 데이터셋
from datasets import load_dataset
from PIL import Image
import os

# 1) 데이터 로드 및 train/val split (원본 train 1502장을 80/20으로)
ds = load_dataset(
    "dwb2023/brain-tumor-image-dataset-semantic-segmentation"
)
train_ds, val_ds, test_ds = (
    ds["train"].train_test_split(test_size=0.2, seed=42)["train"],
    ds["train"].train_test_split(test_size=0.2, seed=42)["test"],
    ds["test"],  # test 스플릿 215장
)

# 2) 저장할 폴더 구조 생성 (train, val, test)
base = "/content/HRNet-Image-Classification/brain_tumor_classification/imagenet/images"
for phase in ("train", "val", "test"):
    for cls in ("no_tumor", "tumor"):
        os.makedirs(f"{base}/{phase}/{cls}", exist_ok=True)

# 3) category_id 로 레이블링 후 저장 함수
def save_split(split_ds, phase):
    for i, item in enumerate(split_ds):
        img = item["image"]
        cid = int(item["category_id"])
        # 1 → no_tumor, 2 → tumor
        label = "tumor"    if cid == 2 else \
                "no_tumor" if cid == 1 else None
        if label is None:
            continue
        img.save(f"{base}/{phase}/{label}/{i}.png")

# 4) 각 스플릿 저장
save_split(train_ds, "train")
save_split(val_ds,   "val")
save_split(test_ds,  "test")

# 5) 결과 확인
print("Train/Tumor   :", len(os.listdir(f"{base}/train/tumor")))
print("Train/NoTumor :", len(os.listdir(f"{base}/train/no_tumor")))
print("Val/Tumor     :", len(os.listdir(f"{base}/val/tumor")))
print("Val/NoTumor   :", len(os.listdir(f"{base}/val/no_tumor")))
print("Test/Tumor    :", len(os.listdir(f"{base}/test/tumor")))
print("Test/NoTumor  :", len(os.listdir(f"{base}/test/no_tumor")))


README.md:   0%|          | 0.00/6.22k [00:00<?, ?B/s]

(…)-00000-of-00001-1d7ef230dfbdad78.parquet:   0%|          | 0.00/113M [00:00<?, ?B/s]

(…)-00000-of-00001-0031debf8df4462b.parquet:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

(…)-00000-of-00001-bce5cef16a5cce11.parquet:   0%|          | 0.00/32.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1502 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/215 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/429 [00:00<?, ? examples/s]

Train/Tumor   : 1240
Train/NoTumor : 1344
Val/Tumor     : 153
Val/NoTumor   : 148
Test/Tumor    : 97
Test/NoTumor  : 118


In [ ]:
# Colab 셀에서
!find . -maxdepth 2 -type f -name "*.yaml"

./experiments/cls_hrnet_w30_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w32_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w40_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w44_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w18_small_v2_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w48_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w18_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
./experiments/cls_hrnet_w18_small_v1_sgd_lr5e-2_wd1e-4_bs32_x100.yaml


In [13]:
# HRNet-w18-small-v1
%cd /content/HRNet-Image-Classification
!cp experiments/cls_hrnet_w18_small_v1_sgd_lr5e-2_wd1e-4_bs32_x100.yaml experiments/cls_hrnet_brain_tumor_small.yaml

/content/HRNet-Image-Classification


In [3]:
# HRNet-w64
%cd /content/HRNet-Image-Classification
!cp experiments/cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml experiments/cls_hrnet_brain_tumor.yaml


/content/HRNet-Image-Classification


In [ ]:
%%bash
sed -i 's|^  ROOT:.*|  ROOT: "/content/brain_tumor_classification"|' \
    experiments/cls_hrnet_brain_tumor.yaml
sed -i 's|^  NUM_CLASSES:.*|  NUM_CLASSES: 2|' \
    experiments/cls_hrnet_brain_tumor.yaml

In [ ]:
%pwd

'/content/HRNet-Image-Classification'

In [ ]:
!ls experiments

cls_hrnet_brain_tumor.yaml
cls_hrnet_w18_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w18_small_v1_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w18_small_v2_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w30_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w32_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w40_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w44_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w48_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml


In [ ]:
!find /content/brain_tumor_classification/images -type d -name ".ipynb_checkpoints" -exec rm -rf {} +

find: ‘/content/brain_tumor_classification/images’: No such file or directory


In [ ]:
%%bash
# brain_tumor_classification 아래 모든 .ipynb_checkpoints 폴더를 찾아 삭제
find /content/HRNet-Image-Classification/brain_tumor_classification \
     -type d -name ".ipynb_checkpoints" \
     -exec rm -rf {} +

In [15]:
from PIL import Image
import os

base_dir = "/content/HRNet-Image-Classification/brain_tumor_classification/imagenet/images"
exts = (".png", ".jpg", ".jpeg", ".bmp")

def is_bad(path):
    try:
        img = Image.open(path)
        img.verify()  # 이미지 파일체크
        return False
    except Exception:
        return True

removed = 0
for root, _, files in os.walk(base_dir):
    for fname in files:
        if fname.lower().endswith(exts):
            p = os.path.join(root, fname)
            if is_bad(p):
                print("Removed corrupted:", p)
                os.remove(p)
                removed += 1

print(f"총 {removed}개 파일 삭제했습니다.")


Removed corrupted: /content/HRNet-Image-Classification/brain_tumor_classification/imagenet/images/train/no_tumor/1948.png
총 1개 파일 삭제했습니다.


In [17]:
!python tools/train.py \
  --cfg experiments/cls_hrnet_brain_tumor.yaml \
  --dataDir "/content/drive/MyDrive/tumor_classfication_data"


=> creating /content/drive/MyDrive/Tumor_w64_classfication4/imagenet/cls_hrnet_brain_tumor
=> creating log/imagenet/cls_hrnet/cls_hrnet_brain_tumor_2025-05-05-13-52
Namespace(cfg='experiments/cls_hrnet_brain_tumor.yaml', modelDir='', logDir='', dataDir='/content/drive/MyDrive/tumor_classfication_data', testModel='')
CfgNode({'OUTPUT_DIR': '/content/drive/MyDrive/Tumor_w64_classfication4/', 'LOG_DIR': 'log/', 'DATA_DIR': '/content/drive/MyDrive/tumor_classfication_data', 'GPUS': (0,), 'WORKERS': 4, 'PRINT_FREQ': 1000, 'AUTO_RESUME': False, 'PIN_MEMORY': True, 'RANK': 0, 'CUDNN': CfgNode({'BENCHMARK': True, 'DETERMINISTIC': False, 'ENABLED': True}), 'MODEL': CfgNode({'NAME': 'cls_hrnet', 'INIT_WEIGHTS': True, 'PRETRAINED': '', 'NUM_JOINTS': 17, 'NUM_CLASSES': 1000, 'TAG_PER_JOINT': True, 'TARGET_TYPE': 'gaussian', 'IMAGE_SIZE': [224, 224], 'HEATMAP_SIZE': [64, 64], 'SIGMA': 2, 'EXTRA': CfgNode({'STAGE1': CfgNode({'NUM_MODULES': 1, 'NUM_RANCHES': 1, 'BLOCK': 'BOTTLENECK', 'NUM_BLOCKS': [4

In [ ]:
%cd /content/HRNet-Image-Classification

/content/HRNet-Image-Classification


In [ ]:
import sys, os

# 레포 루트와 lib 디렉터리를 모듈 탐색 경로에 추가
root = "/content/HRNet-Image-Classification"
lib  = os.path.join(root, "lib")
sys.path.insert(0, root)
sys.path.insert(0, lib)

# 올바르게 추가됐는지 확인
print([p for p in sys.path if "HRNet-Image-Classification" in p])

['/content/HRNet-Image-Classification/lib', '/content/HRNet-Image-Classification']


In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, f1_score, recall_score, jaccard_score

# HRNet 의 config/모델 모듈 import
from config import config, update_config
import models
import argparse, os

# 1) Config 업데이트 (train.py 와 동일한 방식)
parser = argparse.ArgumentParser()
parser.add_argument('--cfg',     type=str,
                    default="experiments/cls_hrnet_brain_tumor_small.yaml")
parser.add_argument('--modelDir',type=str, default="")
parser.add_argument('--logDir',  type=str, default="")
parser.add_argument('--dataDir', type=str,
                    default="/content/HRNet-Image-Classification/brain_tumor_classification")
parser.add_argument('--testModel', type=str, default="")
args = parser.parse_args([])
update_config(config, args)

# 2) 모델 생성 & 체크포인트 로드
model = eval(f"models.{config.MODEL.NAME}.get_cls_net")(config)
ckpt = torch.load(
    "/content/drive/MyDrive/Tumor_w18_Small_Output/imagenet/cls_hrnet_brain_tumor_small/model_best.pth.tar",
    map_location="cuda"
)
state_dict = ckpt.get("state_dict", ckpt)
model.load_state_dict(state_dict)
model = torch.nn.DataParallel(model, device_ids=list(config.GPUS)).cuda().eval()

# 3) Test DataLoader 준비
transform = transforms.Compose([
    transforms.Resize((config.MODEL.IMAGE_SIZE[1], config.MODEL.IMAGE_SIZE[0])),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
test_dir = os.path.join(config.DATASET.ROOT, config.DATASET.TEST_SET)
test_ds     = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(
    test_ds,
    batch_size=config.TEST.BATCH_SIZE_PER_GPU,
    shuffle=False,
    num_workers=config.WORKERS,
    pin_memory=True
)

# 4) 예측 & 메트릭 계산
all_probs, all_preds, all_labels = [], [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.cuda()
        logits = model(imgs)
        probs = torch.softmax(logits, dim=1)[:, 1]  # tumor 확률
        preds = (probs > 0.5).long()
        all_probs .extend(probs.cpu().numpy())
        all_preds .extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

rocauc = roc_auc_score(all_labels, all_probs)
f1     = f1_score    (all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
iou    = jaccard_score(all_labels, all_preds)

print(f"ROC-AUC : {rocauc:.4f}")
print(f"F1      : {f1:.4f}")
print(f"Recall  : {recall:.4f}")
print(f"IoU     : {iou:.4f}")


ROC-AUC : 0.9829
F1      : 0.9040
Recall  : 0.8639
IoU     : 0.8248


In [ ]:
test_dir = os.path.join(config.DATASET.ROOT, "test")
print(test_ds.class_to_idx)      # {'no_tumor': 0, 'tumor': 1}
print(test_ds.classes)           # ['no_tumor','tumor']

{'no_tumor': 0, 'tumor': 1}
['no_tumor', 'tumor']


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

print(classification_report(
    all_labels, all_preds,
    target_names=test_ds.classes,
    digits=4
))

Confusion Matrix:
 [[567  23]
 [ 66 419]]
              precision    recall  f1-score   support

    no_tumor     0.8957    0.9610    0.9272       590
       tumor     0.9480    0.8639    0.9040       485

    accuracy                         0.9172      1075
   macro avg     0.9218    0.9125    0.9156      1075
weighted avg     0.9193    0.9172    0.9167      1075



In [29]:
%cd /content/HRNet-Image-Classification

/content/HRNet-Image-Classification


In [39]:
# Colab 노트북의 맨 위 셀에 한 번만 실행
%env PYTHONPATH=/content/HRNet-Image-Classification:$PYTHONPATH

env: PYTHONPATH=/content/HRNet-Image-Classification:$PYTHONPATH


In [44]:
%%bash
export PYTHONPATH=/content/HRNet-Image-Classification:/content/HRNet-Image-Classification/lib:$PYTHONPATH

python eval_tumor4.py \
  --cfg         /content/HRNet-Image-Classification/experiments/cls_hrnet_brain_tumor.yaml \
  --test-dir    /content/drive/MyDrive/tumor_classfication_data/imagenet/images/Testing \
  --ckpt        /content/drive/MyDrive/Tumor_w64_classfication4/imagenet/cls_hrnet_brain_tumor/model_best.pth.tar \
  --batch-size  32 \
  --num-workers 4



              precision    recall  f1-score   support

      glioma     0.9765    0.9700    0.9732       300
  meningioma     0.9607    0.9575    0.9591       306
     notumor     0.9757    0.9926    0.9841       405
   pituitary     0.9966    0.9833    0.9899       300

    accuracy                         0.9771      1311
   macro avg     0.9774    0.9759    0.9766      1311
weighted avg     0.9772    0.9771    0.9771      1311

Per-Class metrics:
glioma          precision=0.9765  recall=0.9700  f1-score=0.9732  support=300
meningioma      precision=0.9607  recall=0.9575  f1-score=0.9591  support=306
notumor         precision=0.9757  recall=0.9926  f1-score=0.9841  support=405
pituitary       precision=0.9966  recall=0.9833  f1-score=0.9899  support=300


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
